In [1]:
import warnings
warnings.filterwarnings('ignore')
from plotnine import *

from sklearn.decomposition import PCA
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split # simple TT split cv


import numpy as np

In [4]:
spot=pd.read_csv("data.csv")
spot.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [5]:
var=["valence", "year", "acousticness", "danceability", "instrumentalness", "duration_ms","liveness","explicit","key","loudness","mode","popularity","speechiness","tempo"]

X=spot[var]
y=spot["energy"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2)

z= StandardScaler()

X_train[var]=z.fit_transform(X_train[var])
X_test[var]=z.fit_transform(X_test[var])

X_train.head()

,valence,year,acousticness,danceability,instrumentalness,duration_ms,liveness,explicit,key,loudness,mode,popularity,speechiness,tempo
92214,0.250100,1.665006,-0.684777,1.093049,-0.532757,-0.256509,-0.400736,-0.304956,-0.910814,1.146885,0.643533,-1.439561,0.493020,2.121209
110830,1.356400,-1.033292,1.189485,0.480479,-0.522915,-0.283751,0.237902,-0.304956,1.365707,0.936325,0.643533,-1.439561,-0.324080,-0.701583
166642,0.310927,0.894063,-1.255829,1.824731,-0.528012,0.503744,0.557221,-0.304956,-1.195379,0.987647,0.643533,0.895060,0.727535,0.230914
145220,1.337392,-0.339444,-0.764533,-0.863773,-0.510554,-0.420811,-0.144140,-0.304956,-0.626249,0.441737,0.643533,-0.432470,-0.282114,2.716422
55257,-0.905623,1.279535,-0.142437,0.015379,-0.532746,-0.163908,-0.543289,-0.304956,-1.195379,-0.088530,0.643533,1.078167,-0.353703,-0.516237


In [6]:
lr= LinearRegression()

lr.fit(X_train,y_train)

print("Train: ", mean_absolute_error(y_train, lr.predict(X_train)))
print("Test: ", mean_absolute_error(y_test, lr.predict(X_test)))

Train:  0.0936125355655929
Test:  0.09314069461647982


In [29]:
n = 1000 #number of samples per model
nsims = 1000 # number of simulations
alpha = 0.1 # how harshly we penalize

def compare(n = 10000, alpha = 1):
    #linear
    lr = LinearRegression()
    lr.fit(spot[var], spot[y])
    lr_co = lr.coef

    #lasso
    lasso = Lasso(alpha = alpha,
                  fit_intercept = True, normalize = False,
                  tol=0.0000001, max_iter = 100000)
    lasso.fit(spot[var], spot[y])
    lasso_co = lasso.coef_
    
    #ridge
    ridge = Ridge(alpha = n*alpha, fit_intercept = True,
                       normalize = False, tol=0.0000001, max_iter = 100000)
    ridge.fit(spot[var], df[y])
    ridge_co = ridge.coef_


    conames = var
    coefs = np.concatenate([lr_co,lasso_co,ridge_co])

    model = np.repeat(np.array(["Linear", "LASSO", "Ridge"]), [14, 14, 14], axis=0)

    df = pd.DataFrame({"conames": conames,
                      "coefs": coefs,
                      "model": model})


    return(spot)

In [30]:
# running the simulation
sim = compare(n = n, alpha = alpha)

for i in range(1,nsims):
    sim = sim.append(compare(n = n, alpha = alpha), ignore_index = True)

KeyError: "None of [Float64Index([0.21100000000000002,               0.341,               0.166,\n                            0.309,               0.193, 0.34600000000000003,\n                            0.203, 0.08800000000000001,                0.13,\n                            0.257,\n              ...\n                            0.569,  0.6859999999999999,              0.0333,\n              0.47200000000000003,             0.00759,  0.8079999999999999,\n                            0.753,               0.858,               0.623,\n                            0.721],\n             dtype='float64', length=170653)] are in the [columns]"

In [22]:
#put the coefficents in order
sim["conames"] = pd.Categorical(sim["conames"], categories=["valence", "year", "acousticness", "danceability", "instrumentalness", "duration_ms","liveness","explicit","key","loudness","mode","popularity","speechiness","tempo"])

    
# all coefficients
(ggplot(sim, aes(x = "variables",
                y = "coefs",
                fill = "model")) +
  geom_boxplot() + ggtitle("All Coefficients") + theme_minimal())

NameError: name 'sim' is not defined